In [ ]:
import pandas as pd

In [ ]:
# Load the raw data and select only BF2
zscore = True
load_df = pd.read_csv('csv/raw_data_20231113.csv', low_memory=False)
load_df = load_df[load_df['MetaStudy']=='BioFINDER-1']
load_df.info(verbose=True)

In [ ]:
f = [
    'PL_ptau217_pgml_Lilly_imputed_2020',
    'CSF_ptau217_pgml_Lilly_2021',
    'CSF_Ab42_Ab40_ratio_imputed_Elecsys_2020_2022', 'age',
    'apoe_genotype_baseline_variable', 'adas_delayed_word_recall',
    'education_level_years_baseline_variable', 'gender_baseline_variable',
    'cognitive_status_baseline_variable',
    'mmse_score', 'Abnormal_CSF_Ab42_Ab40_Ratio', 
    'diagnosis_baseline_variable',  
    'fnc_ber_com_composite'
    ]
new_fea_names = [
                 'Plasma Lilly P-tau217',
                 'CSF Lilly P-tau217',
                 'CSF Aβ42/Aβ40',
                 'Age',
                 'APOE',
                 'ADAS',
                 'Education',
                 'Sex',
                 'Cognitive status',
                 'MMSE',
                 'CSF Abnormal Ratio',
                 'Diagnosis status',
                 'fnc_ber_com_composite']

In [ ]:
select_df = load_df[f].reset_index(drop=True)
select_df = select_df.set_axis(new_fea_names, axis=1)
select_df.info()

In [ ]:
if zscore:
    stand_df = select_df[['Plasma Lilly P-tau217','CSF Lilly P-tau217']]
    nonstand_df = select_df.drop(['Plasma Lilly P-tau217','CSF Lilly P-tau217'],axis=1)
    z_df = pd.DataFrame()
    features =  ['Cognitive status', 'CSF Abnormal Ratio']
    for f in stand_df.columns:
        cal_df = pd.concat([stand_df[f], nonstand_df[features]], axis=1)
        cal_df = cal_df.dropna(how='any')
        control_df = cal_df[((cal_df['Cognitive status'] == 'Normal') | (cal_df['Cognitive status'] == 'SCD')) & (cal_df['CSF Abnormal Ratio'] == 0)]
        z_mean = control_df[f].mean()
        z_std = control_df[f].std()
        z_df[f] = (stand_df[f]-z_mean)/z_std

    z_df = pd.concat([z_df, nonstand_df],axis=1)
    z_df.to_csv('csv/BF1_R_Z.csv', index=False)
else:
    select_df.to_csv('csv/BF1_R.csv', index=False)

In [ ]:
# # Check the data date range (optional)
# select_df['csf_date'].dropna(how='any').max()